# Material Inventory and Nuclear Tracking Simulator (MINTs) Demonstration Notebook

This notebook contains a high-level guide to running fuel cycle simulations in MINTs.

Simulations are conceptualized as a full fuel cycle, from the mining facility through to the reactor which is generating power.
MINTs currently has implementations for Pressurized Heavy Water Reactor (PHWR) fuel cycles, Light Water Reactor (LWR) fuel cycles, and Prismatic High Temperature Reactor (HTR-Prismatc) fuel cycles.

## Running a PHWR Fuel Cycle Simulation
### Running the fuel cycle
Fuel cycles can be imported from the MINTs `fuel_cycles` module.
The flow of material between facilities is outlined in a json file for each fuel cycle, and these default material flow specifications can serve as a starting point for customization according to the needs of individual users.
A path to a customized fuel cycle specification json file can be passed as the first argument to the fuel cycle simulation initialization methods, and a second argument can be passed to specify the location of a log file for simulation output, if desired.

In [1]:
# Import the PHWR simulation
from mints.fuel_cycles import PHWR

# Create a simulation instance
fuel_cycle = PHWR()

Once the configuration has been specified, the material flow can be simulated by invoking the `run_simulation` method, specifying how long to run the simulation for:

In [2]:
simulation_length_weeks = 104
fuel_cycle.run_simulation(simulation_length_weeks)

Starting simulation
Reactor is online (time = 56)
Simulation complete; Running time=111.24450087547302


### Generating material flow datasets
Now that the material movement has been simulated, we can produce week-by-week records of facilities inventories and inputs/outputs by using the built-in data generation method:

In [3]:
fuel_cycle.write_results('./PHWR_demo_output.h5')

Saved /uranium_mine_1/shipping_mba to PHWR_demo_output.h5
Saved /uranium_mine_1/shipping_mba_ins_and_outs to PHWR_demo_output.h5
Saved /conversion_1/receiving_mba to PHWR_demo_output.h5
Saved /conversion_1/receiving_mba_ins_and_outs to PHWR_demo_output.h5
Saved /conversion_1/shipping_mba to PHWR_demo_output.h5
Saved /conversion_1/shipping_mba_ins_and_outs to PHWR_demo_output.h5
Saved /conversion_1/U235_lost_material to PHWR_demo_output.h5
Saved /conversion_1/U235_lost_material_ins_and_outs to PHWR_demo_output.h5
Saved /conversion_1/U238_lost_material to PHWR_demo_output.h5
Saved /conversion_1/U238_lost_material_ins_and_outs to PHWR_demo_output.h5
Saved /conversion_1/O_lost_material to PHWR_demo_output.h5
Saved /conversion_1/O_lost_material_ins_and_outs to PHWR_demo_output.h5
Saved /conversion_1/U235_in_process to PHWR_demo_output.h5
Saved /conversion_1/U235_in_process_ins_and_outs to PHWR_demo_output.h5
Saved /conversion_1/U238_in_process to PHWR_demo_output.h5
Saved /conversion_1/U238

### Loading material flow datasets
Facility inventory records can be loaded into a python dictionary using the built-in outfile reader:

In [15]:
from mints.utils.data_utils import load_all_inventory
store = load_all_inventory('PHWR_demo_output.h5')
store['conversion_1']['shipping_mba']

,id,weight,when,where,what,form,week
0,43,400.043842,5,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,5
1,44,399.972648,5,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,5
2,45,399.901738,5,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,5
3,46,400.071664,5,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,5
4,47,399.945095,5,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,5
...,...,...,...,...,...,...,...
1879,1376,400.004664,102,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,102
1880,1377,399.891310,102,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,102
1881,1378,399.976427,102,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,102
1882,1379,400.188486,102,conversion_1,"UO2 ({'U235': 0.0061, 'U238': 0.87544, 'O': 0....",drum,102


### Fuel Cycle Attributes

A fuel cycle is composed of individual facilities which material moves through during simulation.
The base PHWR fuel cycle is composed of 4 facilities:

In [5]:
print(fuel_cycle.facilities.keys())

dict_keys(['uranium_mine_1', 'conversion_1', 'fuel_fabrication_1', 'reactor_1'])


Each facility is then further broken down into Material Balance Areas (MBAs) representative of what a realistic safeguards implementation for a facility may look like:

In [6]:
# Print MBAs in the reactor facility
fuel_cycle.facilities['reactor_1'].dict_of_stores().keys()

dict_keys(['receiving_mba', 'spent_fuel_mba', 'core_mba'])

Facilities have additional attributes specified in the fuel cycle json which control the speed at which material moves through the facility, the priority the facility receives when ordering materials from other facilities, the distributions which define the weights and attributes of items, and other parameters:

In [7]:
fuel_cycle.facility_configs['conversion_1']

{'priority': 1,
 'in_material': 'U3O8',
 'out_material': 'UO2',
 'receiving_drum_throughput': 7,
 'receiving_max_inventory': 40,
 'shipping_facility': 'uranium_mine_1',
 'drum_weight_mean_kg': 400,
 'drum_weight_std_kg': 0.1,
 'input_container_type': 'drum',
 'conversion_max_drum_throughput': 10,
 'process_loss': 0,
 'output_container_type': 'drum',
 'declared': True}